In [1]:
import torch
import numpy as np
from torchvision import datasets
import torchvision.transforms as transforms

# check if CUDA is available
train_on_gpu = torch.cuda.is_available()

if not train_on_gpu:
    print('CUDA is not available.  Training on CPU ...')
else:
    print('CUDA is available!  Training on GPU ...')

CUDA is available!  Training on GPU ...


Prepare Train / test data sets

In [2]:
num_workers = 0
batch_size = 512
valid_size = 0.2

train_transform = transforms.Compose([
    transforms.RandomHorizontalFlip(p=0.5),
    transforms.RandomCrop(32, padding=4),
    transforms.ToTensor(),
    transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5)),
])

test_transform = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))
    ])

train_data = datasets.CIFAR10('data', train=True,
                              download=True, transform=train_transform)
test_data = datasets.CIFAR10('data', train=False,
                             download=True, transform=test_transform)

# prepare data loaders (combine dataset and sampler)
train_loader = torch.utils.data.DataLoader(train_data, batch_size=batch_size,
    num_workers=num_workers)
valid_loader = torch.utils.data.DataLoader(test_data, batch_size=batch_size, 
    num_workers=num_workers)
test_loader = torch.utils.data.DataLoader(test_data, batch_size=batch_size, 
    num_workers=num_workers)

# specify the image classes
classes = ['airplane', 'automobile', 'bird', 'cat', 'deer',
           'dog', 'frog', 'horse', 'ship', 'truck']

Files already downloaded and verified
Files already downloaded and verified


In [3]:
from muxcnn.models.ResNetHer import ResNetHer
import torch.nn as nn
from pytorch_lightning import Trainer
from pytorch_lightning.callbacks import ModelCheckpoint, EarlyStopping
from muxcnn.models.lightning import LitModel
import torch.nn.functional as F

/home/hoseung/anaconda3/envs/fhe/lib/python3.10/site-packages/lightning_utilities/core/imports.py:132: Unbuilt egg for bbsQt [unknown version] (/home/hoseung/Work/FHE/Kinect_BBS_demo)
/home/hoseung/anaconda3/envs/fhe/lib/python3.10/site-packages/lightning_utilities/core/imports.py:132: Unbuilt egg for bbsQt [unknown version] (/home/hoseung/Work/FHE/Kinect_BBS_demo)


In [4]:
model = ResNetHer([1,1,1])#, hermite=True)#, activation=None)

model.to('cuda')
train_on_gpu=True

# Instantiate the model and Lightning module
criterion = nn.CrossEntropyLoss()
lit_model = LitModel(model, criterion)

# Define Trainer and fit the model
checkpoint_callback = ModelCheckpoint(monitor='val_loss')
early_stop_callback = EarlyStopping(monitor='val_loss', patience=15)


In [5]:

trainer = Trainer(max_epochs=100, callbacks=[checkpoint_callback, early_stop_callback])
trainer.fit(lit_model, train_dataloaders=train_loader, val_dataloaders=valid_loader)

# Test the model
trainer.test(dataloaders=test_loader)

/home/hoseung/anaconda3/envs/fhe/lib/python3.10/site-packages/lightning_utilities/core/imports.py:132: Unbuilt egg for bbsQt [unknown version] (/home/hoseung/Work/FHE/Kinect_BBS_demo)
GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
HPU available: False, using: 0 HPUs
You are using a CUDA device ('NVIDIA GeForce RTX 4060 Laptop GPU') that has Tensor Cores. To properly utilize them, you should set `torch.set_float32_matmul_precision('medium' | 'high')` which will trade-off precision for performance. For more details, read https://pytorch.org/docs/stable/generated/torch.set_float32_matmul_precision.html#torch.set_float32_matmul_precision
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]

  | Name      | Type             | Params | Mode 
-------------------------------------------------------
0 | model     | ResNetHer        | 77.9 K | train
1 | criterion | CrossEntropyLoss | 0      | train
-------------------------------------------------------
77.9 K    Trainable 

Sanity Checking: |          | 0/? [00:00<?, ?it/s]

/home/hoseung/anaconda3/envs/fhe/lib/python3.10/site-packages/pytorch_lightning/trainer/connectors/data_connector.py:424: The 'val_dataloader' does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` to `num_workers=31` in the `DataLoader` to improve performance.
/home/hoseung/anaconda3/envs/fhe/lib/python3.10/site-packages/pytorch_lightning/trainer/connectors/data_connector.py:424: The 'train_dataloader' does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` to `num_workers=31` in the `DataLoader` to improve performance.


Training: |          | 0/? [00:00<?, ?it/s]

Validation: |          | 0/? [00:00<?, ?it/s]

...

Validation: |          | 0/? [00:00<?, ?it/s]

/home/hoseung/anaconda3/envs/fhe/lib/python3.10/site-packages/pytorch_lightning/trainer/call.py:54: Detected KeyboardInterrupt, attempting graceful shutdown...
/home/hoseung/anaconda3/envs/fhe/lib/python3.10/site-packages/pytorch_lightning/trainer/connectors/checkpoint_connector.py:145: `.test(ckpt_path=None)` was called without a model. The best model of the previous `fit` call will be used. You can pass `.test(ckpt_path='best')` to use the best model or `.test(ckpt_path='last')` to use the last model. If you pass a value, this warning will be silenced.
Restoring states from the checkpoint path at /home/hoseung/Work/FHE/MuxConv/scripts/lightning_logs/version_85/checkpoints/epoch=2-step=294.ckpt
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]
Loaded model weights from the checkpoint at /home/hoseung/Work/FHE/MuxConv/scripts/lightning_logs/version_85/checkpoints/epoch=2-step=294.ckpt
/home/hoseung/anaconda3/envs/fhe/lib/python3.10/site-packages/pytorch_lightning/trainer/connectors/data_connec

Testing: |          | 0/? [00:00<?, ?it/s]

────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────
       Test metric             DataLoader 0
────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────
        test_acc            0.48399999737739563
        test_loss           1.4719175100326538
────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────


[{'test_loss': 1.4719175100326538, 'test_acc': 0.48399999737739563}]

In [9]:
trainer.fit(lit_model, train_dataloaders=train_loader, val_dataloaders=valid_loader)

LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]

  | Name      | Type             | Params | Mode 
-------------------------------------------------------
0 | model     | ResNetHer        | 77.9 K | train
1 | criterion | CrossEntropyLoss | 0      | train
-------------------------------------------------------
77.9 K    Trainable params
0         Non-trainable params
77.9 K    Total params
0.311     Total estimated model params size (MB)


Sanity Checking: |          | 0/? [00:00<?, ?it/s]

Training: |          | 0/? [00:00<?, ?it/s]

Validation: |          | 0/? [00:00<?, ?it/s]

Validation: |          | 0/? [00:00<?, ?it/s]

...

Validation: |          | 0/? [00:00<?, ?it/s]

Validation: |          | 0/? [00:00<?, ?it/s]

Validation: |          | 0/? [00:00<?, ?it/s]

Validation: |          | 0/? [00:00<?, ?it/s]

Validation: |          | 0/? [00:00<?, ?it/s]

In [10]:
trainer.save_checkpoint("smooth_sep_bn3.pt")

In [ ]:
import matplotlib.pyplot as plt 

## Validation accuracy

## 성능 

(Conv - BN - Avg Pool)
conv 5x5, same padding = 67
conv 3x3, same padding = 68
conv 3x3, valid padding = 67



--------------------------

relu + maxpool: ~62%

relu + avgpool: ~58% -- OK, maxpool -> avgpool은 큰 문제 없음. 

approx. relu + avgpool: 52% !! 

approx. relu + avgpool + BN (2Conv + 3FC, 20 epoch): 58% 정도? 

approx. relu + avgpool + BN (2Conv + 2FC, 50 epoch: 59% 


## Number of trainable parameters

In [ ]:
model_parameters = filter(lambda p: p.requires_grad, model.parameters())
params = sum([np.prod(p.size()) for p in model_parameters])
print(f"We have to train {params} parameters")

We have to train 77882 parameters
